<a href="https://colab.research.google.com/github/siddheshtungare/Portfolio-analysis-streamlit/blob/main/Fundamental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamentals Evaluation 

We will run some fundamentals comparison of how the tickers in our portfolio look against the other securities in the whole index

In [1]:
import yfinance as yf
import pandas as pd
import json
import hvplot.pandas

In [2]:
index_to_run = "^AXJO"

## Step1: Get data for all securities for that index. We have stored them in the `Resources/symbols_list.json` file

In [93]:
# Get all tickers for this index
with open('Resources/symbols_list.json', 'r') as json_file:
    symbols_dict = json.load(json_file)

tickers_names = symbols_dict[index_to_run]
# The symbols are read in this format: "NAB.AX: National Australia Bank". 
# We will need to split at the : and take the first part as our ticker codes
tickers_all = [ticker.split(":")[0] for ticker in tickers_names]

In [85]:
# list_of_tickers = ['MSFT', 'AAPL']

string_of_names = ""
for ticker in tickers_all:
    string_of_names += ticker + " "

string_of_names = string_of_names[:-1]
len(string_of_names)

139

In [86]:
tickers = yf.Tickers(string_of_names)

Downloading data takes around 1-2 mins

In [87]:
# Download Fundamentals data for all the tickers
dict_info = [tickers.tickers[ticker].info for ticker in tickers_all]
len(dict_info)

20

In [88]:
list_of_keys = ["overallRisk", "averageVolume", "beta", "marketCap", "ebitda", "forwardPE", "totalRevenue" ]

In [89]:
whatisit = []
for index in range(len(tickers_all)):
    records = {"index": tickers_all[index]}
    records['overallRisk'] = dict_info[index].get('overallRisk')
    records['averageVolume'] = dict_info[index].get('averageVolume')
    records['beta'] = dict_info[index].get('beta')
    records['marketCap'] = dict_info[index].get('marketCap')
    records['ebitda'] = dict_info[index].get('ebitda')
    records['forwardPE'] = dict_info[index].get('forwardPE')
    records['totalRevenue'] = dict_info[index].get('totalRevenue')

    whatisit += [records]

df_fundamentals = pd.DataFrame(whatisit)

In [90]:
print(df_fundamentals.shape)
df_fundamentals.sample()

(20, 8)


,index,overallRisk,averageVolume,beta,marketCap,ebitda,forwardPE,totalRevenue
7,MQG.AX,1,868637,1.132882,66145820672,NaN,14.439662,19140999168


### Data Cleanup 

There maybe some NaNs and some more nasty surprises

Check NaNs

In [ ]:
df_fundamentals.isna().sum()

NaNs present, we'll need to fill them up with some default values: using the mean of the columns values

In [ ]:
for col in list_of_keys: 

    try: 
        df_fundamentals[col].fillna(df_fundamentals[col].mean(), inplace = True)
    except TypeError as err: 
        print(f"Exception TypeError raised for {col}")

df_fundamentals.isna().sum()

Check dtypes

In [ ]:
df_fundamentals.dtypes

`forwardPE` shouldnt be an Object. This means there are some junk values in there

In [ ]:
df_fundamentals.forwardPE.describe()

In [ ]:
df_fundamentals.loc[df_fundamentals.forwardPE == "Infinity"]

**LOL** Price to Earnings ratio = Infinity 🤣

Set the forwardPE ratio to the mean of the column

Then replace the null values with the mean of the column 

In [ ]:
mean_forwardPR = df_fundamentals.loc[df_fundamentals.forwardPE != "Infinity", "forwardPE"].mean()

In [ ]:
df_fundamentals.loc[df_fundamentals.forwardPE == "Infinity", "forwardPE"] = mean_forwardPR
df_fundamentals['forwardPE'].fillna(df_fundamentals['forwardPE'].mean(), inplace = True)

Verify

In [ ]:
df_fundamentals.forwardPE.describe()

#### Data Cleanup complete

Store the cleaned data to a CSV, so that streamlit app can load the DF from it and display the plots

In [ ]:
df_fundamentals.to_csv(f"Resources/Fundamentals_data_{index_to_run}.csv", index=None)

## Read data from excel and proceed with plotting

In [104]:
df_fundamentals = pd.read_csv(f"Resources/Fundamentals_data_{index_to_run}.csv", index_col=0)
df_fundamentals.head(2)

,overallRisk,averageVolume,beta,marketCap,ebitda,forwardPE,totalRevenue
index,,,,,,,
CBA.AX,3.0,2379518.0,0.832582,1.689591e+11,1.093453e+09,17.670753,2.642800e+10
BHP.AX,2.0,7979777.0,0.834973,2.213654e+11,2.611000e+10,11.563492,5.418400e+10


## Start exploring some plots

In [105]:
list_of_keys = ["overallRisk", "averageVolume", "beta", "marketCap", "ebitda", "forwardPE", "totalRevenue" ]

### Distribution plots

In [106]:
plot = df_fundamentals.hvplot.kde("marketCap", title="Normal Distribution")

plot

:Distribution   [marketCap]   (Density)

In [ ]:
plot = df_fundamentals.hvplot.kde("marketCap", title="Normal Distribution")

plot


In [ ]:
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np
import pandas as pd

fig = ff.create_distplot(
    [df_fundamentals["ebitda"].tolist()],
    group_labels=["ebitda"],
    show_hist=False,
).add_traces(
    px.histogram(df_fundamentals, x="ebitda")
    .update_traces(yaxis="y3", name="histogram")
    .data
).update_layout(yaxis3={"overlaying": "y", "side": "right"}, showlegend=False)

fig


In [ ]:
import plotly.express  as px

px.histogram(df_fundamentals, x="averageVolume", histnorm='probability density')


In [ ]:
box_plot = df_fundamentals.hvplot.box(y=['totalRevenue', 'marketCap'], width=500, height=400) #by='totalRevenue', 
box_plot

### New plan - Draw a spider web (radar-chart)

y-axis = list_of_keys

x-axis = percentile score of a security

[https://plotly.com/python/radar-chart/](https://plotly.com/python/radar-chart/)

In [107]:
# Example from the doco

# df = px.data.wind()
# print(df)
# fig = px.scatter_polar(df, r="frequency", theta="direction")
# fig.show()

Given a ticker, create a DF for the percentiles of each value

In [131]:
tickers_to_plot = ["SUL.AX","CBA.AX","WOW.AX"]
ticker_to_plot = "SUL.AX"

In [120]:
for key in list_of_keys:
    # print(key)
    df_fundamentals[f'{key}_percentile'] = df_fundamentals[key].rank(pct=True)

df_fundamentals.head(2)

,overallRisk,averageVolume,beta,marketCap,ebitda,forwardPE,totalRevenue,overallRisk_percentile,averageVolume_percentile,beta_percentile,marketCap_percentile,ebitda_percentile,forwardPE_percentile,totalRevenue_percentile
index,,,,,,,,,,,,,,
CBA.AX,3.0,2379518.0,0.832582,1.689591e+11,1.093453e+09,17.670753,2.642800e+10,0.3475,0.585,0.465,0.99,0.75,0.65,0.965
BHP.AX,2.0,7979777.0,0.834973,2.213654e+11,2.611000e+10,11.563492,5.418400e+10,0.2100,0.910,0.475,1.00,1.00,0.26,0.995


In [132]:
# Given a ticker, lets run the radar chart
# df = df_fundamentals.loc[df_fundamentals.index.isin( tickers_to_plot), [col for col in df_fundamentals.columns.to_list() if "percentile" in col] ].T               # Only plot the percentile columns need 
df = df_fundamentals.loc[df_fundamentals.index == ticker_to_plot, [col for col in df_fundamentals.columns.to_list() if "percentile" in col] ].T               # Only plot the percentile columns need 

# df = df_fundamentals.loc[df_fundamentals['index'].str.contains("percentile")]
# df = df[[col for col in df_fundamentals.columns.to_list() if "percentile" in col]]           # Only plot the percentile columns need 

df


index,SUL.AX
overallRisk_percentile,0.645
averageVolume_percentile,0.225
beta_percentile,0.785
marketCap_percentile,0.345
ebitda_percentile,0.520
forwardPE_percentile,0.295
totalRevenue_percentile,0.670


In [134]:
import plotly.express as px
fig = px.line_polar(df, r=ticker_to_plot, theta=df.index, line_close=True)
fig.show()


c:\Users\siddh\anaconda3\envs\dev_fintech_course\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

